# Filtering epigenetic data from PacBio Sequencing
## 0. HEADER
### 0.1. Load libraries 

In [1]:
import subprocess
from multiprocessing import Pool

import pandas as ours
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import motifs
from itertools import product

### 0.2. General functions

In [2]:
def ambiguous_dna_list(seq):
    """return list of all possible sequences given an ambiguous DNA input"""
    d = ambiguous_dna_values
    return list(map("".join, product(*map(d.get, seq))))

## 1. Loading DATA

In [3]:
# Loading a strain genome (12_016)
genome_12_016 = SeqIO.parse(open("../Vaestuarianus_GENOME/VAESTLR12016.1-Contigs.fasta"),'fasta')

# Reading the PacBio DATA and storing it in a dataframe
df_PACBIO = ours.read_csv(r'../Differences_epi_12-016_02-041_07-115/Methylation_Vaestu_alt.csv')


# Selecting rows with non-null values for Score_016 and keeping only the first 11 columns
df_12_016 = df_PACBIO.loc[~df_PACBIO['Score_016'].isna(), df_PACBIO.columns[0:11]]

# Selecting rows with non-null values for Score_115 and keeping columns 1-7 and 12-15
df_07_115 = df_PACBIO.loc[~df_PACBIO['Score_115'].isna(), df_PACBIO.columns[[*range(0,7), *range(11,15)]]]

# Selecting rows with non-null values for Score_041 and keeping columns 1-7 and 16-19
df_02_041 = df_PACBIO.loc[~df_PACBIO['Score_041'].isna(), df_PACBIO.columns[[*range(0,7), *range(15,19)]]]

print(df_12_016.head(20))

       seqid  position                                    context type strand  \
7   contig_1       975  TGGGTGGTAATTAACCTCAGATCGGTTTAAGCCATCGCCAT  m6A      -   
8   contig_1      1247  GATGGCGATGGCTTAAACCGATCTGAGGTTAATTAATAGGT  m6A      +   
9   contig_1      1248  AACCTATTAATTAACCTCAGATCGGTTTAAGCCATCGCCAT  m6A      -   
10  contig_1      1548  AATATCGTACCGTTATTGAGAATATTACTGCTGGTTCAAAT  m6A      +   
11  contig_1      1556  ATGATGTCATTTGAACCAGCAGTAATATTCTCAATAACGGT  m6A      -   
12  contig_1      1759  AGTTCACAATGAGCTCTCAGATCGTAGCCATGCTAATGGTC  m6A      +   
13  contig_1      1760  TGACCATTAGCATGGCTACGATCTGAGAGCTCATTGTGAAC  m6A      -   
14  contig_1      2680  TGTAGACCGAGTAGATTGGGATCGACGAGATGAGGAACTTG  m6A      +   
15  contig_1      2681  ACAAGTTCCTCATCTCGTCGATCCCAATCTACTCGGTCTAC  m6A      -   
16  contig_1      3546  TTACCATGGCAAGGTGTTTGATCCTGCTTGTGGTAGTGGTG  m6A      +   
17  contig_1      3547  CCACCACTACCACAAGCAGGATCAAACACCTTGCCATGGTA  m6A      -   
18  contig_1      3621  CCAT

In [4]:
_#### - 1.1. Separate in multiple DATA-FRAME the motifs for a strain
motif_GATC_12_016 = df_12_016[(df_12_016["motif"] == "GATC") | (df_12_016["motif"] == "GATC (GARANNNNNNCTG,GATC)")].iloc[:, 0:3]
motif_CAGNNNNNNTYTC_12_016 = df_12_016[df_12_016["motif"] == "CAGNNNNNNTYTC"].iloc[:, 0:3]
motif_GARANNNNNNCTG_12_016 = df_12_016[(df_12_016["motif"] == "GARANNNNNNCTG") | (df_12_016["motif"] == "GARANNNNNNCTG (ACCNNNNNNNTTCY,GARANNNNNNCTG)")].iloc[:, 0:3]
motif_GGWCC_12_016 = df_12_016[df_12_016["motif"] == "GGWCC"].iloc[:, 0:3]
motif_GTAYNNNNGTTA_12_016 = df_12_016[df_12_016["motif"] == "GTAYNNNNGTTA"].iloc[:, 0:2]
motif_TAACNNNNRTAC_12_016 = df_12_016[df_12_016["motif"] == "TAACNNNNRTAC"].iloc[:, 0:2]
motif_RGAANNNNNNNGGT_12_016 = df_12_016[df_12_016["motif"] == "RGAANNNNNNNGGT"].iloc[:, 0:2]
motif_ACCNNNNNNNTTCY_12_016 = df_12_016[(df_12_016["motif"] == "ACCNNNNNNNTTCY") | (df_12_016["motif"] == "ACCNNNNNNNTTCY,RGAANNNNNNNGGT")].iloc[:, 0:2]

### 1.1. Separate in multiple DATA-FRAME the motifs for a strain

In [5]:
#### - 1.1. Separate in multiple DATA-FRAME the motifs for a strain
#### - 1.2. Creating a function to automatise the processus of exporting the coordinates data for each motifs
def export_motifs(df, strain_name):
    print(df.head())
    motif_GATC = df[(df["motif"] == "GATC") | (df["motif"] == "GATC (GARANNNNNNCTG,GATC)")][["position", "context"]]
    motif_CAGNNNNNNTYTC = df[df["motif"] == "CAGNNNNNNTYTC"][["position", "context"]]
    motif_GARANNNNNNCTG = df[(df["motif"] == "GARANNNNNNCTG") | (df["motif"] == "GARANNNNNNCTG (ACCNNNNNNNTTCY,GARANNNNNNCTG)")][["position", "context"]]
    motif_GGWCC = df[df["motif"] == "GGWCC"][["position", "context"]]
    motif_GTAYNNNNGTTA = df[df["motif"] == "GTAYNNNNGTTA"][["position", "context"]]
    motif_TAACNNNNRTAC = df[df["motif"] == "TAACNNNNRTAC"][["position", "context"]]
    motif_RGAANNNNNNNGGT = df[df["motif"] == "RGAANNNNNNNGGT"][["position", "context"]]
    motif_ACCNNNNNNNTTCY = df[(df["motif"] == "ACCNNNNNNNTTCY") | (df["motif"] == "ACCNNNNNNNTTCY,RGAANNNNNNNGGT")][["position", "context"]]
  
    # We calculate and add the end position of the motif
    motif_GATC["end_pos"] = motif_GATC["position"] + 4
    motif_CAGNNNNNNTYTC["end_pos"] = motif_CAGNNNNNNTYTC["position"] + 13
    motif_GARANNNNNNCTG["end_pos"] = motif_GARANNNNNNCTG["position"] + 13
    motif_GGWCC["end_pos"] = motif_GGWCC["position"] + 5
    motif_GTAYNNNNGTTA["end_pos"] = motif_GTAYNNNNGTTA["position"] + 12
    motif_TAACNNNNRTAC["end_pos"] = motif_TAACNNNNRTAC["position"] + 12
    motif_RGAANNNNNNNGGT["end_pos"] = motif_RGAANNNNNNNGGT["position"] + 14
    motif_ACCNNNNNNNTTCY["end_pos"] = motif_ACCNNNNNNNTTCY["position"] + 14
  
    # We can then export those dataframes
    motif_GATC.to_csv(f"./MOTIFS/motif_GATC_{strain_name}.txt", sep=" ", header=False, index=False, quoting=None)
    motif_CAGNNNNNNTYTC.to_csv(f"./MOTIFS/motif_CAGNNNNNNTYTC_{strain_name}.txt", sep=" ", header=False, index=False, quoting=None)
    motif_GARANNNNNNCTG.to_csv(f"./MOTIFS/motif_GARANNNNNNCTG_{strain_name}.txt", sep=" ", header=False, index=False, quoting=None)
    motif_GGWCC.to_csv(f"./MOTIFS/motif_GGWCC_{strain_name}.txt", sep=" ", header=False, index=False, quoting=None)
    motif_GTAYNNNNGTTA.to_csv(f"./MOTIFS/motif_GTAYNNNNGTTA_{strain_name}.txt", sep=" ", header=False, index=False, quoting=None)
    motif_TAACNNNNRTAC.to_csv(f"./MOTIFS/motif_TAACNNNNRTAC_{strain_name}.txt", sep=" ", header=False, index=False, quoting=None)
    motif_RGAANNNNNNNGGT.to_csv(f"./MOTIFS/motif_RGAANNNNNNNGGT_{strain_name}.txt", sep=" ", header=False, index=False, quoting=None)
    motif_ACCNNNNNNNTTCY.to_csv(f"./MOTIFS/motif_ACCNNNNNNNTTCY_{strain_name}.txt", sep=" ", header=False, index=False, quoting=None)



## 2. Finding coordinates of context sequences in a genome to find true coordinates of methylations marks
### 2.1.

In [33]:
nb_motifs = motif_CAGNNNNNNTYTC_12_016.shape[0]
df_motifs = ours.DataFrame({
    'motif': [''] * nb_motifs,
    'context': [''] * nb_motifs,
    'pos_start': [0] * nb_motifs,
    'pos_end': [0] * nb_motifs,
    'contig': [2] * nb_motifs
})
for i_motif in range(nb_motifs):
    motif_pos_tmp = subprocess.check_output(['../0_bin/Finding_context_of_motif',
                                              '../Vaestuarianus_GENOME/Vaestlr12016-contigs.fasta',
                                              motif_CAGNNNNNNTYTC_12_016['context'][i_motif]]).decode().strip()

    if len(motif_pos_tmp) != 0:
        motif_pos_contig = motif_pos_tmp.split(' ')
        df_motifs.at[i_motif, 'pos_start'] = motif_pos_contig[0]
        df_motifs.at[i_motif, 'contig'] = motif_pos_contig[1]

KeyError: 0

In [28]:
motifs_list = df_12_016["context"].values.tolist()
process_list= ours.DataFrame({
    "Position": ours.Series(dtype = "int"),
    "Contig" : ours.Series(dtype = "int")
})
process_list_tmp  = process_list

process_str = ""

Motifs_context = ' '.join(motifs_list)
for i_motifs in range(10):
    process = subprocess.run(['../0_bin/Finding_context_of_motif', '../Vaestuarianus_GENOME/Vaestlr12016-contigs.fasta',motifs_list[i_motifs]], 
                            capture_output=True,
                            text = True)
    process_str = process.stdout
    process_tmp = process_str.split()
    print(process_tmp[0])
    process_list_tmp["Position"].loc["1"] = int(process_tmp[0])
    process_list_tmp["Contig"].loc["1"] = int(process_tmp[1])
    print(process_list_tmp)
    process_list = process_list.append(process_list_tmp)
    
print(process_list)

3044384
Empty DataFrame
Columns: [Position, Contig]
Index: []
990339
Empty DataFrame
Columns: [Position, Contig]
Index: []
3044111
Empty DataFrame
Columns: [Position, Contig]
Index: []
990640
Empty DataFrame
Columns: [Position, Contig]
Index: []
3043804
Empty DataFrame
Columns: [Position, Contig]
Index: []
990851
Empty DataFrame
Columns: [Position, Contig]
Index: []
3043600
Empty DataFrame
Columns: [Position, Contig]
Index: []
991772
Empty DataFrame
Columns: [Position, Contig]
Index: []
9373
Empty DataFrame
Columns: [Position, Contig]
Index: []
992638
Empty DataFrame
Columns: [Position, Contig]
Index: []
Empty DataFrame
Columns: [Position, Contig]
Index: []


In [7]:
#find_methylation_and_export(df_12_016, '12_016')
'''
CAGNNNNNNTYTC

GARANNNNNNCTG
GARANNNNNNCTG (ACCNNNNNNNTTCY,GARANNNNNNCTG)

GTAYNNNNGTTA

ACCNNNNNNNTTCY
ACCNNNNNNNTTCY,RGAANNNNNNNGGT
RGAANNNNNNNGGT
RGAANNNNNNNGGT (ACCNNNNNNNTTCY,RGAANNNNNNNGGT)

TAACNNNNRTAC
'''

len("AATTGACTCAGGCGTACTG")

19

In [8]:
# Reading the CSV file and storing it in a dataframe
df_PACBIO = ours.read_csv("../Differences_epi_12-016_02-041_07-115/Methylation_Vaestu_alt.csv", header=0)

# Selecting rows with non-null values for Score_016 and keeping only the first 11 columns
df_12_016 = df_PACBIO.loc[~df_PACBIO['Score_016'].isna(), df_PACBIO.columns[0:11]]

# Selecting rows with non-null values for Score_115 and keeping columns 1-7 and 12-15
df_07_115 = df_PACBIO.loc[~df_PACBIO['Score_115'].isna(), df_PACBIO.columns[[*range(0,7), *range(11,15)]]]

# Selecting rows with non-null values for Score_041 and keeping columns 1-7 and 16-19
df_02_041 = df_PACBIO.loc[~df_PACBIO['Score_041'].isna(), df_PACBIO.columns[[*range(0,7), *range(15,19)]]]

# Separating motifs for strain 12_016 and calculating the end position of each motif
motif_GATC_12_016 = df_12_016.loc[(df_12_016['motif'] == 'GATC') | (df_12_016['motif'] == 'GATC (GARANNNNNNCTG,GATC)'), df_12_016.columns[0:3]].copy()
motif_CAGNNNNNNTYTC_12_016 = df_12_016.loc[df_12_016['motif'] == 'CAGNNNNNNTYTC', df_12_016.columns[0:3]].copy()
motif_GARANNNNNNCTG_12_016 = df_12_016.loc[(df_12_016['motif'] == 'GARANNNNNNCTG') | (df_12_016['motif'] == 'GARANNNNNNCTG (ACCNNNNNNNTTCY,GARANNNNNNCTG)'), df_12_016.columns[0:3]].copy()
motif_GGWCC_12_016 = df_12_016.loc[df_12_016['motif'] == 'GGWCC', df_12_016.columns[0:3]].copy()
motif_GTAYNNNNGTTA_12_016 = df_12_016.loc[df_12_016['motif'] == 'GTAYNNNNGTTA', df_12_016.columns[0:2]].copy()
motif_TAACNNNNRTAC_12_016 = df_12_016.loc[df_12_016['motif'] == 'TAACNNNNRTAC', df_12_016.columns[0:2]].copy()
motif_RGAANNNNNNNGGT_12_016 = df_12_016.loc[df_12_016['motif'] == 'RGAANNNNNNNGGT', df_12_016.columns[0:2]].copy()
motif_ACCNNNNNNNTTCY_12_016 = df_12_016.loc[(df_12_016['motif'] == 'ACCNNNNNNNTTCY') | (df_12_016['motif'] == 'ACCNNNNNN

SyntaxError: EOL while scanning string literal (3314683429.py, line 21)